# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [228]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        s = np.ones(len(l_s)) - np.sum((((l_c + r_c) / (l_s + r_s)) ** 2), axis=1)
        s_l = np.ones(len(l_s)) - np.sum(((l_c / l_s) ** 2), axis=1)
        s_r = np.ones(len(l_s)) - np.sum(((r_c / r_s) ** 2), axis=1)
        l_s = l_s.reshape((-1))
        r_s = r_s.reshape((-1))
        return s - l_s / (l_s + r_s) * s_l - r_s / (l_s + r_s) * s_r
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        s = - np.sum((((l_c + r_c) / (l_s + r_s)) * np.log2((l_c + r_c) / (l_s + r_s))), axis=1)
        s_l = - np.sum(((l_c / l_s) * np.log2(l_c / l_s)), axis=1)
        s_r = - np.sum(((r_c / r_s) * np.log2(r_c / r_s)), axis=1)
        l_s = l_s.reshape((-1))
        r_s = r_s.reshape((-1))
        return s - l_s / (l_s + r_s) * s_l - r_s / (l_s + r_s) * s_r 

    def __misclass(self, l_c, l_s, r_c, r_s):
        #массив приростов для каждого разбиения
        l_s = l_s.reshape((-1))
        r_s = r_s.reshape((-1))
        s = np.ones(len(l_s)) - (l_c + r_c).max(axis=1) / l_s + r_s
        s_l = np.ones(len(l_s)) - (l_c).max(axis=1) / l_s
        s_r = np.ones(len(r_s)) - (r_c).max(axis=1) / r_s
        return s - l_s / (l_s + r_s) * s_l - r_s / (l_s + r_s) * s_r 
    # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]# Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature)]# Ваш код

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)# Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # сортируем выборку по значению признака вычисляем кол-во классов 
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # вычисляем позиции на которых текущий класс отличается  от следующего
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # eq_el_count - кол-во прдряд идущих объектов 1 класса
        # далее построили матрицу которая описывает сколько элементов какого класса вошло в соответствующее разбиение
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        #это мы подсчитали сколько объектов какого класса попадет в левое поддерево,
        #и в правое поддерево соответственно и посчитали сколько объектов в левом и правом поддереве
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # вычисляем изменение неопределенности для всех разбиений и выбирем лучшее
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # выбираем границу на основе лучшей меры неопределенности и возвращаем ее и саму меру
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        classes = np.bincount(y) / len(y)
        if depth == self.max_depth or len(y) <= self.min_samples_split or (classes >= self.sufficient_share).any(): 
            self.tree[node_id] = [self.LEAF_TYPE, np.argmax(classes)]
            return
            
        features = self.get_feature_ids(x.shape[1])
        mings = -1
        fid = -1
        threshold = -1
        for f in features:
            gs, threshold = self.__find_threshold(x.T[f], y)
            if mings == -1 or mings > gs:
                fid = f
        
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, fid, threshold)
                
        
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, fid, threshold]
        try:
            self.__fit_node(x_left, y_left,  2 * node_id + 1, depth + 1, pred_f)
        except:
            self.tree[node_id] = [self.LEAF_TYPE, np.argmax(classes)]
            return
        try:
            self.__fit_node(x_right, y_right,  2 * node_id + 2, depth + 1, pred_f)
        except:
            self.tree.pop(2 * node_id + 1)
            self.tree[node_id] = [self.LEAF_TYPE, np.argmax(classes)]
            return
        
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [23]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [111]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])
x

array([[  0.76612661,  45.        ,   2.        , ...,   6.        ,
          0.        ,   2.        ],
       [  0.95715102,  40.        ,   0.        , ...,   0.        ,
          0.        ,   1.        ],
       [  0.65818014,  38.        ,   1.        , ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [  0.29974515,  44.        ,   0.        , ...,   1.        ,
          0.        ,   2.        ],
       [  0.        ,  30.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.85028295,  64.        ,   0.        , ...,   2.        ,
          0.        ,   0.        ]])

In [229]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)#, criterion='entropy')
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [232]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.355999946594
1.19400000572


## Проверка качества работы

In [115]:
gkf = KFold(n_splits=5, shuffle=True)

In [230]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.929949280785
0.928909952607
0.933399850337
0.930572877692
0.929738494159


In [231]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.890288517502
0.89161885757
0.889041323688
0.889498628087
0.892487423606
